In [ ]:
# Copyright (c) 2022 Neil Majithia
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

In [ ]:
%matplotlib inline
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import os
import pycel
from pycel import ExcelCompiler
import pandas as pd
import gc


filename = r'simple simulator unemployment.xlsx'
plt.style.use('seaborn-poster')
display(widgets.HTML('''<style>
    p, h1, h2, h3, h4, ul, ol, li { max-width: 70vw !important; margin: 0px auto 5px auto !important;}
    img {max-width:  !important; display: block; margin: 0px auto 1px auto !important;}
    p, ul, ol { font-size: 1.4em !important}
    .widget-label { min-width: min-content !important; }
    /* .jp-RenderedHTMLCommon.jp-RenderedMarkdown.jp-MarkdownOutput { height: min-content; min-height: 35px;} */
</style>'''))
style = {'description_width': 'initial'}

<h1 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    Understanding Inflation and Central Bank Policy Choices
</h1>

In [ ]:
excel = ExcelCompiler(filename=filename,cycles=True)

In [ ]:
shock_size = excel.evaluate('simulator!L7')
no_policy = excel.evaluate('simulator!A1:J30')
before_policy = excel.evaluate('simulator!A1:J6')
after_policy = excel.evaluate('simulator!A1:J7')
# to display use display(); for dataframe have to save and open with pandas

In [ ]:
def PlotStuff(exceloutputs:list, title, after = False, show=True, auto=False, legend=False, shockline=True, shocktext=False, recoverytext=False):
    '''Plots inflation and unemployment on side-by-side subplots'''
    colourlist = ['#f05a5b', 'grey', 'pink']
    figs, axs = plt.subplots(1, 4, figsize=[24, 8])
    for data in exceloutputs:
        colour = colourlist[exceloutputs.index(data)]
        if after:
            axs[0].plot([data[i][0] for i in range(1, len(data)-1)], [data[i][4] for i in range(1, len(data)-1)], colour)
            axs[1].plot([data[i][0] for i in range(1, len(data)-1)], [data[i][3] for i in range(1, len(data)-1)], colour)
            axs[2].plot([data[i][0] for i in range(1, len(data)-1)], [100*data[i][-1] for i in range(1, len(data)-1)], colour)
            axs[3].plot([data[i][0] for i in range(1, len(data)-2)], [data[i][-3] for i in range(1, len(data)-2)], colour)
            axs[0].plot([data[-2][0], data[-1][0]], [data[-2][4], data[-1][4]], 'black')
            axs[1].plot([data[-2][0], data[-1][0]], [data[-2][3], data[-1][3]], 'black')
            axs[2].plot([data[-2][0], data[-1][0]], [100*data[-2][-1], 100*data[-1][-1]], 'black')
            axs[3].plot([data[-3][0], data[-2][0]], [data[-3][-3], data[-2][-3]], 'black')
            axs[3].set_xlim(left=0.75, right=6.25)

        else:
            axs[0].plot([data[i][0] for i in range(1, len(data))], [data[i][4] for i in range(1, len(data))], colour)
            axs[1].plot([data[i][0] for i in range(1, len(data))], [data[i][3] for i in range(1, len(data))], colour)
            axs[2].plot([data[i][0] for i in range(1, len(data))], [100*data[i][-1] for i in range(1, len(data))], colour)
            axs[3].plot([data[i][0] for i in range(1, len(data)-1)], [data[i][-3] for i in range(1, len(data)-1)], colour)
            axs[0].plot(data[-1][0] + 1, data[-1][4])
            axs[1].plot(data[-1][0] + 1, data[-1][3])
            axs[2].plot(data[-1][0] + 1, 100*data[-1][-1])
            axs[3].plot(data[-1][0] + 1, data[-1][-3])
    axs[0].set_title('Inflation %')
    axs[1].set_title('GDP')
    axs[2].set_title('Unemployment %')
    axs[3].set_title('Real Interest Rate %')
    axs[0].set_xlabel('Period')
    axs[1].set_xlabel('Period')
    axs[2].set_xlabel('Period')
    axs[3].set_xlabel('Period')
    if shockline:
        axs[0].vlines(x=4, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[1].vlines(x=4, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[2].vlines(x=4, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[3].vlines(x=4, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[0].vlines(x=5, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[1].vlines(x=5, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[2].vlines(x=5, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
        axs[3].vlines(x=5, ymin=0, ymax=200, color='silver', linestyles='dashed', label='shock')
    if shocktext:
        axs[0].text(x=3.85, y=8.16, s='SHOCK', rotation='vertical', horizontalalignment='center')
        axs[1].text(x=3.85, y=100.675, s='SHOCK', rotation='vertical', horizontalalignment='center')
        axs[2].text(x=3.85, y=11.175, s='SHOCK', rotation='vertical', horizontalalignment='center')
        axs[3].text(x=3.85, y=7.675, s='SHOCK', rotation='vertical', horizontalalignment='center')
    if recoverytext:
            axs[0].text(x=5.3, y=8.15, s='POLICY\nREACTION', rotation='vertical', horizontalalignment='center')
            axs[1].text(x=5.3, y=100.625, s='POLICY\nREACTION', rotation='vertical', horizontalalignment='center')
            axs[2].text(x=5.3, y=11.125, s='POLICY\nREACTION', rotation='vertical', horizontalalignment='center')
            axs[3].text(x=5.3, y=7.625, s='POLICY\nREACTION', rotation='vertical', horizontalalignment='center')


    if not auto:
        axs[0].set_ylim(bottom=1.5, top=9)
        axs[1].set_ylim(bottom=93.5, top=101.5)
        axs[2].set_ylim(bottom=4, top=12)
        axs[3].set_ylim(bottom=0.5, top=8.5)


    figs.suptitle(title, fontsize=16)
    
    if legend:
        label1 = mlines.Line2D([], [], color='#f05a5b', label='Your Choice')
        label2 = mlines.Line2D([], [], color='grey', label='Benchmark')
        axs[0].legend(handles=[label1, label2], bbox_to_anchor=(0.8, 1.2))
        # axs[0].legend(handles=[label1, label2])

    if show:
        plt.show()
    else:
        return figs, axs

def AppendtoPlot(figs, axs, new_excel):
    axs[0].plot([new_excel[i][0] for i in range(1, len(new_excel))], [new_excel[i][4] for i in range(1, len(new_excel))])
    axs[1].plot([new_excel[i][0] for i in range(1, len(new_excel))], [new_excel[i][3] for i in range(1, len(new_excel))])
    axs[2].plot([new_excel[i][0] for i in range(1, len(new_excel))], [new_excel[i][-1] for i in range(1, len(new_excel))])
    axs[3].plot([new_excel[i][0] for i in range(1, len(new_excel))], [new_excel[i][-3] for i in range(1, len(new_excel))])
    return figs, axs
    


October 2022: The job of central banks in these years is like riding a rollercoaster. After having to fight with deflationary forces arising from the financial crisis in the late 2000’s and from the 2020-2021 COVID-19 pandemic, inflation became the number one public enemy during 2022. Several shocks that range from the disruption in the global production chain to increasing commodity prices due to the Ukraine war have increased CPI inflation to levels incompatible with central banks’ mandate of price stability. (see Figure 1)

<img src='intrograph.jpg'>
Figure 1
Source: 

Central bankers may yearn for past too-low inflation scenarios where, if successful, monetary policy could yield both price stability and lower unemployment. Instead, when negative supply shocks - like an oil shock - hit the economy central bankers face a nasty trade-off between price stability and unemployment, at least in the short run. Bringing inflation down to hit the target in the short run is technically possible but, is it worth the economic downturn and the increase in unemployment that this would entail? Most central banks consider that it is not since they take the inflation target as a medium run goal. Within this time frame central banks face two uncertainties that condition their decision. 
* First, how persistent are the underlying shocks and/or may there be in the near future counterbalancing disinflationary shocks? 
* Second, how anchored are inflation expectations to the central bank’s inflation target and how serious is the danger of second round effects? 
A tight disinflationary policy could produce needless suffering if the underlying inflationary forces dissipate in the near future. However, a too timid response of monetary policy to inflation may trigger de-anchoring and second round effects that would push inflation further away from the target.

This interactive simulation can help you to understand some of the monetary policy dilemmas faced by central banks in the wake of the pandemic and the war in Ukraine explained above. 
* All the sections share the common feature of representing an economy hit by an inflationary supply shock (think of an oil price shock). 
* First, as a benchmark, the consequences of the shock for inflation are  shown assuming that monetary policy is passive and that expected inflation is equal to last period’s inflation, i.e.inflation expectations are not anchored to the central bank’s inflation target. . 
* In the second section you take charge of monetary policy and use your interest rate choice to  change the consequences of the shock for the first period after the shock. Here you will experience the short run trade-off implied in this monetary policy decision. 
* In the third section you can compare the no-policy reaction benchmark with two alternative monetary policy paths: an active but too timid monetary policy and one that succeeds in reducing inflation. . 
* In the fourth section you can put yourself in the skin of central banks with different degrees of inflation aversion to see that optimal monetary policy is compatible with different degrees of tightening. 
* Finally, the last section introduces inflation anchoring for the first time. It reveals why anchoring inflation expectations is so important to central banks because it allows monetary policy to bring inflation back to the target quicker and with lower unemployment costs.


<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    The Supply Shock
</h2>

When the price of imported energy and or commodities goes up, certain goods such as fuel or food become more expensive for the consumer. Costs of production go up as well and firms try to pass-through these higher costs to final prices. These effects happen relatively quickly and concentrate in the first period, as shown in the figure below (Fig. 2). What happens in the following periods depends on policy. In the following sections you can explore different policy options and scenarios.

<img src="supplyshockintrograph.jpg">
Figure 2

<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    No Policy Reaction
</h2>

<h3>If there is no policy response to the rise in inflation caused by the oil shock,  the situation quickly spirals out of control as inflation rises period by period (Fig. 3)</h3>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [12, 6])
ax.plot([no_policy[i][0] for i in range(1, 16)], [no_policy[i][4] for i in range(1, 16)], '#f05a5b')
ax.set_title('An inflation shock with no policy response')
ax.set_ylabel('Inflation %')
ax.set_xlabel('Period')
ax.xaxis.set_ticks(range(1, 16))

plt.show()

Figure3

When the price of oil increases, inflation goes up in the shock period due to the increase in the price of fuel for consumers and the pass-through of the increase in firms’ costs to final prices. This increase in inflation has two effects relevant for next period:

* First, it reduces the purchasing power of workers, which means that workers’ real wages are below the level consistent with the state of the labour market. [The real wage shown by the PS curve falls below the real wage on the WS curve. i.e., the PS curve shifts down.]
* Second, the increase in inflation increases inflation expectations for the future. [The PC curve shifts up.]

Due to these two effects, in the next period nominal wages increase faster both to offset the loss in purchasing power due to the unexpected increase in past inflation and to avoid erosion of purchasing power due to higher expected inflation. Wage setters are frustrated in the attempt to restore the real wage to its initial level [on the WS curve]. Firms pass on the increase in unit labour costs arising from the rise in nominal wages to final prices to sustain their profit margin, increasing inflation above expected inflation. Therefore, in each period thereafter the attempt to restore workers’ purchasing power will maintain nominal wage growth and inflation above expected inflation, fueling an inflation spiral.

<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    You choose: Monetary Policy Reaction to an Inflation Shock 
</h2>

<h3>There's been a permanent inflation shock in period 5.</h3> 


<p>Pretend you're the Central Bank - use the slider to choose a real interest rate for the next period to try and stabilise inflation, and note how your choice affects the rest of the economy too. </p>

In [ ]:
out = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
display(out)

Fig 4

In [ ]:
r = widgets.IntSlider(1, 1, 7, description='Post-Shock Interest Rate')
sub = widgets.Button(description='Submit')
reset = widgets.Button(description='Reset')


with out:
    out.clear_output(wait=True)
    display(widgets.HBox((r, sub)))
    # display(before_policy)
    PlotStuff([before_policy], title='', shocktext=True, recoverytext=True, auto=False)


def on_sub(b):
    excel.evaluate('simulator!H6')
    chosenrate = r.value
    excel.set_value('simulator!H6', chosenrate)
    excel.recalculate()
    after_policy = excel.evaluate('simulator!A1:J7')
    with out:
        out.clear_output(wait=True)
        display(widgets.HBox((r, sub, reset)))
        display(widgets.HTML(f'<p>You chose a rate of {chosenrate}% - see below for its effects.</p>'))
        # display(excel.evaluate('simulator!H6'))
        # display(after_policy)
        PlotStuff([after_policy], title='', after=True, shocktext=True, recoverytext=True, auto=False)
        if chosenrate == 4:
            display(widgets.HTML('<h3>You have stabilised inflation - congratulations.</h3>'))
            display(widgets.HTML('<p>This is because you have chosen an interest rate equal to the stabilizing real interest rate [on the IS curve at the new lower equilibrium level of output]. However, your success in stopping the inflation spiral has been at the expense of an increase in unemployment. Increasing the interest rate you have reduced aggregate demand. With lower demand for final goods and services firms reduce labour demand, which increases unemployment. Higher unemployment reduces the workers\' reservation wage and makes wage setters accept a fall in the real wage, although of lower magnitude than the erosion in real wage caused by the oil shock. [This is a move along the WS curve.] Now the wage-setting real wage exactly matches the real wage consistent with firms\' price-setting behaviour and the higher oil price [on the lower PS curve]. Therefore, after your policy choice nominal wage growth and therefore actual inflation are equal to expected and past inflation. This is why inflation remains constant. [The PC curve does not shift upward.]</p>'))
            display(widgets.HTML('<h4>You have stabilized inflation but unemployment is higher and inflation, although stable, is higher than the central bank\'s target, which we assume was the pre-existing rate of inflation.</h4>'))
            print()
        elif 1 <= chosenrate < 4:
            display(widgets.HTML('<h3>Inflation is still getting higher, and meanwhile unemployment is getting worse too! </h3>'))
            display(widgets.HTML('<p>This is because you have chosen an interest rate lower than the stabilizing level. Therefore, you have not stopped the inflation spiral, although it proceeds more slowly after your policy choice. [The stabilizing real interest rate has gone up because equilibrium output is now lower.]</p>'))
            display(widgets.HTML('<p>By increasing the interest rate, you have reduced aggregate demand. With lower demand for final goods and services, firms reduce labour demand, which increases unemployment. Higher unemployment reduces the workers\' reservation wage and makes wage setters accept a fall in the real wage, although of lower magnitude than the erosion in real wage caused by the oil shock. [This is a move along the WS curve.] Therefore, after the policy choice the wage-setting real wage is still higher than the real wage consistent with firms\' price-setting behaviour [the wage on the WS is still above that on the on the lower PS curve] and conditions in the labour market meant that wage setters still seek to increase the purchasing power of workers. Thus, they set nominal wage growth (and effectively, inflation) above expected and past inflation. This is why inflation is still on an upwards path, although a more moderate one due to the partial acceptance as a result of higher unemployment of the dent made in real wages by the shock.</p>'))
            display(widgets.HTML('<h4>With your policy choice you have driven the economy to a stagflation scenario, that is, a situation that combines an increase in both inflation and unemployment.</h4>'))
            print()
        elif 4 < chosenrate < 7:
            display(widgets.HTML('<h3>Inflation is heading back down to the central bank\'s target. This is because you have chosen an interest rate higher than the stabilizing level. However, employment is tanking due to your policy choice.</h3>'))
            display(widgets.HTML('<p>Increasing the interest rate you have reduced aggregate demand. With lower demand for final goods and services, firms reduce labour demand, which increases unemployment. Higher unemployment reduces the workers\' reservation wage and makes wage setters accept a fall in the real wage [a move along the WS curve]. This effect is so strong that now the wage-setting real wage is lower than the real wage consistent with firms\' price-setting behaviour and the higher oil price [on the lower PS curve]. The acceptance – due to the deterioration in the labour market – of further reductions in the real wage translates into setting nominal wage growth (and effectually inflation) below expected and past inflation. This is why inflation falls [the PC shifts down].</p>'))
            display(widgets.HTML('<h4>Despite your policy having reduced inflation, this is still higher than its pre-shock value. In the following periods you would need to keep the interest rate above its stabilizing level to drive inflation back to its pre-shock value (the central bank\'s target). [the PC would keep shifting down period by period]</h4>'))
            print()
        elif chosenrate == 7:
            display(widgets.HTML('<h3>You have chosen an interest rate higher than the equilibrium level and enough to drive inflation back to its pre-shock value. However, the cost of your policy shock in terms of unemployment is the highest.</h3>'))
            display(widgets.HTML('<p>Increasing the interest rate you have reduced aggregate demand. With lower demand for final goods and services firms reduce labour demand, which increases unemployment. Higher unemployment reduces the workers\' reservation wage and makes wage setters accept a fall in the real wage [a move along the WS curve]. This effect is so strong that now the wage-setting real wage is lower than the real wage consistent with firms\' price-setting behaviour and the higher oil price [on the lower PS curve]. The acceptance – due to the deterioration in the labour market – of further reductions in the real wage translates into setting nominal wage growth (and effectually inflation) below expected and past inflation. This is why inflation falls [the PC shifts down].</p>'))
            display(widgets.HTML('<h4>Since you have hit the pre-shock inflation value (the central bank\'s target), then in the following period you would just need to adjust the interest rate to its equilibrium level. This is a \'shock therapy\' approach to the problem.</h4>'))
            print()

def on_reset(b):
    with out:
        out.clear_output(wait=True)
        display(widgets.HBox((r, sub)))
        # display(before_policy)
        PlotStuff([before_policy], title='', shocktext=True, recoverytext=True, auto=False)

sub.on_click(on_sub)
reset.on_click(on_reset)


<h3 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
   Test Yourself
</h3>

1. You can verify that choosing the appropriate interest rate, the central bank manages to bring the inflation back to target. But could the central bank do it even better, in terms of inflation? Discuss if the central bank could avoid the increase of inflation in the shock period. Why or why not?
2. Answer the following questions:

&ensp;&ensp;a. Choosing different values for the interest rate, try to gauge the sensitivity of GDP with respect to the interest rate and the slope coefficient for the Phillips curve (hint: both are integers).
    
&ensp;&ensp;b. There is evidence that the Phillips curve has flattened during the lasts decades (see, for instance, <a href='https://www.ecb.europa.eu/press/key/date/2021/html/ecb.sp211007~ab617e7d60.en.html'>Prospects for inflation: sneezes and breezes (europa.eu)</a>). Discuss how the flattening of the Phillips curve may affect the cost of bringing inflation down after the supply shock.

<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
   Comparing Different Reactions by the Central Bank to an Oil Shock 
</h2>

<h3>In response to an inflation shock, a Central Bank could have different approaches to recovery - each with their own costs and benefits over both short and long term.</h3>

Use Figure 5 to explore the whole path of the economy following different policy choices.Look first at the path of the real interest rate to see what the Central Bank does; then look at the outcomes for the economy.


In [ ]:
excel1 = ExcelCompiler(filename=filename,cycles=True)
excel1.evaluate('simulator!H6')
excel1.set_value('simulator!H6', 1)
excel1.recalculate()
no_change_policy = excel1.evaluate('simulator!A1:J30')
excel2 = ExcelCompiler(filename='simple sim endog cb.xlsx', cycles=True)
timid_policy = excel2.evaluate('timid!A1:K30')
optimal_policy = excel2.evaluate('optimal!A1:K30')


In [ ]:
out2 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
display(out2)

Figure 5

In [ ]:
title = widgets.HTML('<h2> Comparing types of central banks </h2>')
dropdown = widgets.Dropdown(options=['Select an option from below', 'CB that wants to avoid higher unemployment despite equilibrium change',
 'CB that slowly accepts change to unemployment', 
 'CB that accepts costly recession and unemployment below equilibrium to prioritise inflation'], value='Select an option from below', layout={'width': 'max-content'}, description='')

def get_plots(CB):
    if CB == 'CB that wants to avoid higher unemployment despite equilibrium change':
        PlotStuff([no_change_policy[:15]], title='CB that wants to avoid change to unemployment despite equilibrium change', after=False, auto=False)
        display(widgets.HTML('<h2>Leaving policy unchanged is not sustainable</h2>'))
        display(widgets.HTML('<p>As shown previously, an unchanged interest rate ensures an unsustainable inflation spiral.</p>'))
    elif CB == 'CB that slowly accepts change to unemployment':
        PlotStuff([timid_policy[:15]], title='CB that slowly accepts change to unemployment',after=False)
        display(widgets.HTML('<h2>A slow response has short term benefits with a soft landing, at the expense of permanently high inflation</h2>'))
        display(widgets.HTML('<p>In this case, the central bank slowly adjusts from below the policy rate to its equilibrium level. The increases in the interest rate are not enough to drive inflation down since the interest rate is always below its stabilizing value. As a result, inflation keeps increasing. Eventually, inflation stabilizes at a level above its pre-shock value when the interest rate hits its equilibrium value. At this point, high inflation is entrenched by higher inflation expectations resulting from a prolonged period of excess inflation.</p>'))
        display(widgets.HTML('<p>It can be seen that in the short run this policy option may seem attractive, since it implies that neither inflation nor unemployment is very high. However, in the medium run it implies both excess inflation and higher unemployment.</p>'))
    elif CB == 'CB that accepts costly recession and unemployment below equilibrium to prioritise inflation':
        PlotStuff([optimal_policy[:15]], title='CB that accepts costly recession and unemployment below equilibrium to prioritise inflation', after=False)
        display(widgets.HTML('<h2>A costly recession, engineered by the CB with high interest rates, brings inflation under control but could be </h2>'))
        display(widgets.HTML('<p>In this case, the central bank responds quickly and strongly to the oil shock, increasing the interest rate above its equilibrium value from the outset. The short run sacrifice in terms of unemployment is severe, but it is necessary if the central bank wants to wipe out the effects of the shock on inflation. Doing this, the central bank avoids a permanent increase in inflation expectations, which allows a relaxation of monetary policy in the medium run to the new equilibrium at higher unemployment and a higher stabilizing interest rate. </p>'))
    else:
        display(widgets.HTML('<h3>Choose a CB behaviour from the Dropdown to see its performance, benefits and costs</h3>'))


with out2:
    out2.clear_output(wait=True)
    display(title)
    widgets.interact(get_plots, CB=dropdown)
    # display(inter)
    # PlotStuff(no_change_policy[:15], title='CB that wants to avoid change to unemployment despite equilibrium change', after=False)
    # PlotStuff(timid_policy[:15], title='CB that slowly accepts change to unemployment',after=False)
    # PlotStuff(optimal_policy[:15], title='CB that accepts costly recession and unemployment below equilibrium to prioritise inflation', after=False)


In [ ]:
qout2 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
qout2

In [ ]:
df2 = pd.read_excel('questionsexcel.xlsx', index_col='id').iloc[0].to_frame().transpose()
qs2 = {}
ans2 = {}
msgs2 = {}

for i in df2.index:
    qs2[i] = [df2.loc[i]['question'], df2.loc[i]['c1'], df2.loc[i]['c2'], df2.loc[i]['c3']]
    ans2[i] = [df2.loc[i]['a1'], df2.loc[i]['a2'], df2.loc[i]['a3']]
    msgs2[i] = [df2.loc[i]['message1'], df2.loc[i]['message2'], df2.loc[i]['message3']]

wlist2 = {}
for q in qs2:
    wlist2[q] = [qs2[q][0]]
    for i in range(1, 4):
        wlist2[q].append(widgets.Checkbox(description=qs2[q][i], style=style, layout=widgets.Layout(min_width='min-content')))

qout2.clear_output()
sub2 = widgets.Button(description='Check my answers')
tryagain2 = widgets.Button(description='Try Again')


with qout2:
    for q in wlist2:
        display(widgets.HTML(value=f'<h4>{wlist2[q][0]}</h4>'))
        for i in wlist2[q][1:]:
            display(i)
        print()
    display(sub2)


def on_sub2(b):
    global ans2
    global msgs2
    qout2.clear_output(wait=True)
    messages = {}
    correct = [None, None, None]
    answers = []
    for q in wlist2:
        messages[q] = []
        for i in wlist2[q][1:]:
            answer = i.value
            answers.append(answer)
        for i in range(1, 4):
            answer = wlist2[q][i].value
            if answer == ans2[q][i-1]:
                messages[q].append(msgs2[q][i-1])
                correct[i-1] = True
            elif answer != ans2[q][i-1]:
                messages[q].append(msgs2[q][i-1])
                correct[i-1] = False    
    with qout2:
        # print('sub2mitted')
        for q in wlist2:
            display(widgets.HTML(value=f'<h3>{wlist2[q][0]}</h3>'))
            for i in range(1, 4):
                display(wlist2[q][i])
                # if messages[q][i-1][0] == 'CORRECT':
                #     display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p><div style="font-weight: bold; color: green">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
                # else:
                #     display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p><div style="font-weight: bold; color: red">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
            # print(messages[q])
            for i in range(3):
                if correct[i] == False:
                    if ans2[q][i] == True:
                        #user did not select, but answer is True
                        notall = True
                    else:
                        #user chose true, answer is false
                        notall = False
                        break
            if correct == [True, True, True]:
                # display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p>CORRECT</p><p>{messages[q]}</p></div>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: green">CORRECT</p>'))
                for i in range(3):
                    if ans2[q][i] == True:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))
            elif notall:
                display(widgets.HTML(f'<p style="font-weight: bold;">You haven\'t selected all the correct answers</p>'))
            else:
                # display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p>INCORRECT</p>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: red">INCORRECT</p>'))
                for i in range(3):
                    if correct[i] == False and ans2[q][i] == False:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))

                # display(widgets.HTML('</div>'))
        display(tryagain2)
        

sub2.on_click(on_sub2)
tryagain2.on_click(on_sub2)

In [ ]:
qout3 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
qout3

In [ ]:
df3 = pd.read_excel('questionsexcel.xlsx', index_col='id').iloc[1].to_frame().transpose()
qs3 = {}
ans3 = {}
msgs3 = {}

for i in df3.index:
    qs3[i] = [df3.loc[i]['question'], df3.loc[i]['c1'], df3.loc[i]['c2'], df3.loc[i]['c3']]
    ans3[i] = [df3.loc[i]['a1'], df3.loc[i]['a2'], df3.loc[i]['a3']]
    msgs3[i] = [df3.loc[i]['message1'], df3.loc[i]['message2'], df3.loc[i]['message3']]

wlist3 = {}
for q in qs3:
    wlist3[q] = [qs3[q][0]]
    for i in range(1, 4):
        wlist3[q].append(widgets.Checkbox(description=qs3[q][i], style=style, layout=widgets.Layout(min_width='min-content')))

qout3.clear_output()
sub3 = widgets.Button(description='Check my answers')
tryagain3 = widgets.Button(description='Try Again')


with qout3:
    for q in wlist3:
        display(widgets.HTML(value=f'<h4>{wlist3[q][0]}</h4>'))
        for i in wlist3[q][1:]:
            display(i)
        print()
    display(sub3)


def on_sub3(b):
    global ans3
    global msgs3
    qout3.clear_output(wait=True)
    messages = {}
    correct = [None, None, None]
    answers = []
    for q in wlist3:
        messages[q] = []
        for i in wlist3[q][1:]:
            answer = i.value
            answers.append(answer)
        for i in range(1, 4):
            answer = wlist3[q][i].value
            if answer == ans3[q][i-1]:
                messages[q].append(msgs3[q][i-1])
                correct[i-1] = True
            elif answer != ans3[q][i-1]:
                messages[q].append(msgs3[q][i-1])
                correct[i-1] = False    
    with qout3:
        # print('sub3mitted')
        for q in wlist3:
            display(widgets.HTML(value=f'<h3>{wlist3[q][0]}</h3>'))
            for i in range(1, 4):
                display(wlist3[q][i])
                # if messages[q][i-1][0] == 'CORRECT':
                #     display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p><div style="font-weight: bold; color: green">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
                # else:
                #     display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p><div style="font-weight: bold; color: red">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
            # print(messages[q])
            for i in range(3):
                if correct[i] == False:
                    if ans3[q][i] == True:
                        #user did not select, but answer is True
                        notall = True
                    else:
                        #user chose true, answer is false
                        notall = False
                        break
            if correct == [True, True, True]:
                # display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p>CORRECT</p><p>{messages[q]}</p></div>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: green">CORRECT</p>'))
                for i in range(3):
                    if ans3[q][i] == True:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))
            elif notall:
                display(widgets.HTML(f'<p style="font-weight: bold;">You haven\'t selected all the correct answers</p>'))
            else:
                # display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p>INCORRECT</p>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: red">INCORRECT</p>'))
                for i in range(3):
                    if correct[i] == False and ans3[q][i] == False:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))

                # display(widgets.HTML('</div>'))
        display(tryagain3)
        

sub3.on_click(on_sub3)
tryagain3.on_click(on_sub3)

<h3 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
   Test Yourself
</h3>

1. Consider the following words taken from a 2022 speech by Jerome Powell, chairman of the US Federal Reserve (https://www.federalreserve.gov/newsevents/speech/powell20220826a.htm): <i>"Reducing inflation is likely to require a sustained period of below-trend growth. Moreover, there will very likely be some softening of labor market conditions. While higher interest rates, slower growth, and softer labor market conditions will bring down inflation, they will also bring some pain to households and businesses. These are the unfortunate costs of reducing inflation. But a failure to restore price stability would mean far greater pain.(…) During the 1970s, as inflation climbed, the anticipation of high inflation became entrenched in the economic decision making of households and businesses. The more inflation rose, the more people came to expect it to remain high, and they built that belief into wage and pricing decisions. (…) History shows that the employment costs of bringing down inflation are likely to increase with delay, as high inflation becomes more entrenched in wage and price setting. The successful Volcker disinflation in the early 1980s followed multiple failed attempts to lower inflation over the previous 15 years. A lengthy period of very restrictive monetary policy was ultimately needed to stem the high inflation and start the process of getting inflation down to the low and stable levels that were the norm until the spring of last year. Our aim is to avoid that outcome by acting with resolve now."</i>

After reading the text, answer the following questions:

a. What do you think Mr. Powell means with <i>"softening of labor market conditions"</i>?

b. Which of the two active policy scenarios ("CB that slowly accepts change to unemployment" and "CB that accepts costly recession and unemployment below equilibrium to prioritise inflation") better fits the 1970’s experience described by Powell? Why?

c. Which of the two active policy scenarios better fits Jerome Powell’s intentions in 2022? Why?

d. Consider scenario number 3 where the central bank quickly reacts to the shock. You can see that in this scenario the interest rate and the unemployment rate peak to 5,5% and 9%, respectively. Now consider scenario number 2 and let us imagine that in period 15 the central bank resolves to tighten monetary policy in order to bring inflation back to the target. How tight should monetary policy be compared to scenario 3 (more/less)? How much unemployment would it cause compared to scenario 3 (more/less)? Why? Which part of the text can help you to answer these questions?


<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    A Central Bank with an Explicit Loss Function and Inflation Target (of 2%): The Policy Maker's Aversion to Inflation
</h2>

<h3>Now that it's clear how a costly recession is necessary, we introduce the variable beta. Beta reflects the extent to which the Central Bank prioritises inflation control over growth/employment.
</h3> 

In the previous section we saw that an oil shock implies a trade-off for the central bank between inflation and unemployment. In this section we show that the way this trade-off is dealt with depends on the central bank’s degree of inflation aversion. [The CB uses the MR curve to determine its interest rate reaction to the shock.]

A benchmark is set where the central bank places the same weight on deviations of inflation from its target and of unemployment from equilibrium. [beta is equal to one in the CB’s loss function]


In [ ]:
out3 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
display(out3)

Fig 6

In [ ]:
out3.clear_output()
def betafunc(slider, show_benchmark):
    # global figs, axs
    excel2.evaluate('optimal!M3')
    chosenrate = slider.value
    excel2.set_value('optimal!M3', chosenrate)
    excel2.recalculate()
    after_policy = excel2.evaluate('optimal!A1:K30')
    if chosenrate < 1:
        message = "With lower values of beta, price stability becomes less important. Therefore, the central bank implements a weaker monetary policy reaction, which has a lesser effect on unemployment at the expense of a slower correction in excess inflation."
    elif chosenrate > 1:
        message = "With a value for beta higher than the benchmark price stability becomes more important. The central bank wants to curb excess inflation quicker, which requires a larger increase in the interest rate and, therefore, a higher short run sacrifice in terms of unemployment."
    else:
        message = ''
    if show_benchmark:
        PlotStuff([after_policy[:15], optimal_policy[:15]], title='', after=False, auto=False, legend=True)
        display(widgets.HTML(f'<h3>{message}</h3>'))
    else:
        PlotStuff([after_policy[:15]], title='', after=False, auto=False)
        display(widgets.HTML(f'<h3>{message}</h3>'))


beta = widgets.FloatSlider(1, min=0.25, max=4, step=0.25, description='CB beta')
betabenchmark = widgets.Checkbox(value=False, description='Show benchmark (beta=1)')
betasub = widgets.Button(description='Submit')
# outtest = widgets.interactive_output(betafunc, {'slider': beta, 'show_benchmark': benchmark})

with out3:
    out3.clear_output()
    display(widgets.HBox((beta, betasub)))
    display(betabenchmark)
    betafunc(beta, betabenchmark.value)

def on_beta_sub(b):
    with out3:
        out3.clear_output()
        display(widgets.HBox((beta, betasub)))
        display(betabenchmark)
        betafunc(beta, betabenchmark.value)

betasub.on_click(on_beta_sub)



In [ ]:
qout4 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
qout4

In [ ]:
df4 = pd.read_excel('questionsexcel.xlsx', index_col='id').iloc[2].to_frame().transpose()
qs4 = {}
ans4 = {}
msgs4 = {}

for i in df4.index:
    qs4[i] = [df4.loc[i]['question'], df4.loc[i]['c1'], df4.loc[i]['c2'], df4.loc[i]['c3']]
    ans4[i] = [df4.loc[i]['a1'], df4.loc[i]['a2'], df4.loc[i]['a3']]
    msgs4[i] = [df4.loc[i]['message1'], df4.loc[i]['message2'], df4.loc[i]['message3']]

wlist4 = {}
for q in qs4:
    wlist4[q] = [qs4[q][0]]
    for i in range(1, 4):
        wlist4[q].append(widgets.Checkbox(description=qs4[q][i], style=style, layout=widgets.Layout(min_width='min-content')))

qout4.clear_output()
sub4 = widgets.Button(description='Check my answers')
tryagain4 = widgets.Button(description='Try Again')


with qout4:
    for q in wlist4:
        display(widgets.HTML(value=f'<h4>{wlist4[q][0]}</h4>'))
        for i in wlist4[q][1:]:
            display(i)
        print()
    display(sub4)


def on_sub4(b):
    global ans4
    global msgs4
    qout4.clear_output(wait=True)
    messages = {}
    correct = [None, None, None]
    answers = []
    for q in wlist4:
        messages[q] = []
        for i in wlist4[q][1:]:
            answer = i.value
            answers.append(answer)
        for i in range(1, 4):
            answer = wlist4[q][i].value
            if answer == ans4[q][i-1]:
                messages[q].append(msgs4[q][i-1])
                correct[i-1] = True
            elif answer != ans4[q][i-1]:
                messages[q].append(msgs4[q][i-1])
                correct[i-1] = False    
    with qout4:
        # print('sub4mitted')
        for q in wlist4:
            display(widgets.HTML(value=f'<h3>{wlist4[q][0]}</h3>'))
            for i in range(1, 4):
                display(wlist4[q][i])
                # if messages[q][i-1][0] == 'CORRECT':
                #     display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p><div style="font-weight: bold; color: green">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
                # else:
                #     display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p><div style="font-weight: bold; color: red">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
            # print(messages[q])
            for i in range(3):
                if correct[i] == False:
                    if ans4[q][i] == True:
                        #user did not select, but answer is True
                        notall = True
                    else:
                        #user chose true, answer is false
                        notall = False
                        break
            if correct == [True, True, True]:
                # display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p>CORRECT</p><p>{messages[q]}</p></div>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: green">CORRECT</p>'))
                for i in range(3):
                    if ans4[q][i] == True:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))
            elif notall:
                display(widgets.HTML(f'<p style="font-weight: bold;">You haven\'t selected all the correct answers</p>'))
            else:
                # display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p>INCORRECT</p>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: red">INCORRECT</p>'))
                for i in range(3):
                    if correct[i] == False and ans4[q][i] == False:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))

                # display(widgets.HTML('</div>'))
        display(tryagain4)
        

sub4.on_click(on_sub4)
tryagain4.on_click(on_sub4)

<h3 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
   Test Yourself
</h3>

Compare the monetary policy reaction to the shock by two central banks with very different beta (for instance, beta=4 compared with the benchmark). Explain in words why with higher beta:

a. The peaks of the interest rate and of unemployment are higher

b. The central bank relaxes policy quicker and unemployment falls quicker to its new equilibrium. (hint: it has to do with inflation expectations)

<h2 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
    Experimenting with Different Inflation Expectation Formation Processes: Anchoring vs Inertia
</h2>

<h3>Inflation recovery is not just dependent on the Central Bank's reaction. </h3> 

Households and firms' expectations about inflation affect their behaviour, and their own reaction to changing interest rates. The way these expectations are formed, and the proportion of the population who share the same expectations, shapes the economy's recovery from a shock

To this point, inflation expectations have excluded any anchoring to the central bank’s target. Anchoring of inflation expectations relaxes the short run trade-off between inflation and unemployment that the central bank faces when dealing with an inflation shock.

The case with no anchoring is set as a benchmark and it matches the cases considered in previous sections [standard PC with expected inflation equal to last period’s inflation].


In [ ]:
excel3 = ExcelCompiler('simple sim endog cb inflation expectations.xlsx', cycles=True)

In [ ]:
out4 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
display(out4)

In [ ]:
out4.clear_output()
def credfunc(slider, show_benchmark):
    # global figs, axs
    excel3.evaluate('optimal!M8')
    chosenrate = slider
    excel3.set_value('optimal!M8', chosenrate)
    excel3.recalculate()
    after_policy = excel3.evaluate('optimal!A1:K30')
    if show_benchmark:
        PlotStuff([after_policy[:15], optimal_policy[:15]], title='', after=False, auto=False, legend=True)
    else:
        PlotStuff([after_policy[:15]], title='', after=False, auto=False)


cred = widgets.FloatSlider(0, min=0, max=1, step=0.25, description='Level of Anchoring')
credbenchmark = widgets.Checkbox(value=False, description='Show benchmark (No Anchoring)')
credsub = widgets.Button(description='Submit')
# outcred = widgets.interactive_output(credfunc, {'slider': cred, 'show_benchmark': benchmark})


with out4:
    out4.clear_output()
    display(widgets.HBox((cred, credsub)))
    display(credbenchmark)
    credfunc(cred.value, credbenchmark.value)

def on_cred_sub(b):
    with out4:
        out4.clear_output()
        display(widgets.HBox((cred, credsub)))
        display(credbenchmark)
        credfunc(cred.value, credbenchmark.value)
    
credsub.on_click(on_cred_sub)

With higher anchoring, inflation expectations are less affected by the temporary upsurge in inflation. Therefore, the central bank needs to increase the interest rate to a lesser extent to achieve their aim of calming inflation. This results in both a lower short run increase in unemployment and a faster return of inflation to its pre-shock level. In the extreme case when there is full anchoring, the central bank just needs to adjust the interest rate to its new equilibrium level from the very onset in order to drive inflation back to its pre-shock level.


In [ ]:
qout5 = widgets.Output(layout={'border': '2px solid gray', 'padding': '5px 5px 5px 5px', 'margin': '0 auto 0 auto', 'max_width': '70vw'})
qout5

In [ ]:
df5 = pd.read_excel('questionsexcel.xlsx', index_col='id').iloc[3].to_frame().transpose()
qs5 = {}
ans5 = {}
msgs5 = {}

for i in df5.index:
    qs5[i] = [df5.loc[i]['question'], df5.loc[i]['c1'], df5.loc[i]['c2'], df5.loc[i]['c3']]
    ans5[i] = [df5.loc[i]['a1'], df5.loc[i]['a2'], df5.loc[i]['a3']]
    msgs5[i] = [df5.loc[i]['message1'], df5.loc[i]['message2'], df5.loc[i]['message3']]

wlist5 = {}
for q in qs5:
    wlist5[q] = [qs5[q][0]]
    for i in range(1, 4):
        wlist5[q].append(widgets.Checkbox(description=qs5[q][i], style=style, layout=widgets.Layout(min_width='min-content')))

qout5.clear_output()
sub5 = widgets.Button(description='Check my answers')
tryagain5 = widgets.Button(description='Try Again')


with qout5:
    for q in wlist5:
        display(widgets.HTML(value=f'<h4>{wlist5[q][0]}</h4>'))
        for i in wlist5[q][1:]:
            display(i)
        print()
    display(sub5)


def on_sub5(b):
    global ans5
    global msgs5
    qout5.clear_output(wait=True)
    messages = {}
    correct = [None, None, None]
    answers = []
    for q in wlist5:
        messages[q] = []
        for i in wlist5[q][1:]:
            answer = i.value
            answers.append(answer)
        for i in range(1, 4):
            answer = wlist5[q][i].value
            if answer == ans5[q][i-1]:
                messages[q].append(msgs5[q][i-1])
                correct[i-1] = True
            elif answer != ans5[q][i-1]:
                messages[q].append(msgs5[q][i-1])
                correct[i-1] = False    
    with qout5:
        # print('sub5mitted')
        for q in wlist5:
            display(widgets.HTML(value=f'<h3>{wlist5[q][0]}</h3>'))
            for i in range(1, 4):
                display(wlist5[q][i])
                # if messages[q][i-1][0] == 'CORRECT':
                #     display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p><div style="font-weight: bold; color: green">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
                # else:
                #     display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p><div style="font-weight: bold; color: red">{messages[q][i-1][0]}</div>{messages[q][i-1][1]}</p></div>'))
            # print(messages[q])
            for i in range(3):
                if correct[i] == False:
                    if ans5[q][i] == True:
                        #user did not select, but answer is True
                        notall = True
                    else:
                        #user chose true, answer is false
                        notall = False
                        break
            if correct == [True, True, True]:
                # display(widgets.HTML(f'<div style="background-color: rgb(161, 230, 161); max-width: fit-content;"><p>CORRECT</p><p>{messages[q]}</p></div>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: green">CORRECT</p>'))
                for i in range(3):
                    if ans5[q][i] == True:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))
            elif notall:
                display(widgets.HTML(f'<p style="font-weight: bold;">You haven\'t selected all the correct answers</p>'))
            else:
                # display(widgets.HTML(f'<div style="background-color: rgb(231, 151, 151); max-width: fit-content;"><p>INCORRECT</p>'))
                display(widgets.HTML(f'<p style="font-weight: bold; color: red">INCORRECT</p>'))
                for i in range(3):
                    if correct[i] == False and ans5[q][i] == False:
                        display(widgets.HTML(f'<p>{messages[q][i]}</p>'))

                # display(widgets.HTML('</div>'))
        display(tryagain5)
        

sub5.on_click(on_sub5)
tryagain5.on_click(on_sub5)

<h3 style="color: #f05a5b; font-family: 'Trebuchet MS', 'Lucida Sans Unicode', sans-serif; font-weight: bold;">
   Test Yourself
</h3>

1. Choose different levels of anchoring and notice how anchoring improves macroeconomic outcomes of monetary policy. Explain with your own words why central banks like a high level of anchoring. Speculate about how a central bank can improve the degree of anchoring.
2. Consider the case of full anchoring (level of anchoring=1). See that the central bank increases interest rate to its new equilibrium value equal 4% and inflation falls back to the target in just one period. Now, in this context of full anchoring, speculate about the consequences for inflation and unemployment of not changing the interest rate after the shock…

&ensp;&ensp;a. …first, considering that full anchoring remains.
    
&ensp;&ensp;b. …second, discuss whether full anchoring may be in danger if the central bank persists in its passive monetary policy and the future implications for inflation.

<p style="font-size:  x-small !important;">Copyright (c) 2022 Neil Majithia </p>

In [ ]:
gc.collect()